In [1]:
import sys
from pathlib import Path

In [2]:
project_root = Path().resolve().parent
sys.path.append(str(project_root))

In [6]:
from src.data.data_loader import DataLoader

In [10]:
data_loader = DataLoader()
qa_pairs = data_loader.load_data()

In [15]:
qa_pairs[0]['qa_pairs']

[{'question': 'what was the percentage change in the net cash from operating activities from 2008 to 2009',
  'answer': '14.1%'}]

In [3]:
from src.data.models.answer_processor import AnswerProcessor


processor = AnswerProcessor()

In [8]:
llm_response = """

{
  "formula": "divide(subtract(9362.2, 9244.9), 9244.9)",
  "formatting_instructions": {
    "prefix": "",
    "suffix": "%",
    "rounding": 2,
    "multiplier": 100
  }
}

"""
    
try:
    answer = processor.process_answer(llm_response)
    print(f"Processed answer: {answer}")  # Should output: "1.27%"
except ValueError as e:
    print(f"Error: {e}")

Processed answer: 1.27%
